In [1]:
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import GloVe
import torchtext.vocab as vocab
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
import numpy as np
import re
import torch.nn as nn
import torch.optim as optim

c:\Users\sfip2\OneDrive\Documents\Christina\UW Machine Learning\ML-ClimateStatementSorting\UW-ML\Lib\site-packages\torchtext\data\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
c:\Users\sfip2\OneDrive\Documents\Christina\UW Machine Learning\ML-ClimateStatementSorting\UW-ML\Lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)

In [4]:
# # Load data from CSV file
# data = pd.read_csv("input/twitter_sentiment_data.csv")

# # Define features
# message_text = data["message"]
# sentiment_value = (data["sentiment"]>0.5).astype(int)

In [5]:
# # Tokenization
# tokenizer = get_tokenizer("basic_english")  # Basic tokenizer for English text
# tokenized_data = [tokenizer(text.lower()) for text in message_text]

In [6]:
# def preprocess_text_sklearn(text):

#     # Define a regular expression pattern to match "RT" at the beginning of the string followed by any characters up to ":"
#     pattern = r'^RT.*?:'

#     # Use re.sub() to replace the matched pattern with an empty string
#     text = re.sub(pattern, '', text)
    
#     # Remove URLs
#     text = re.sub(r'http\S+', '', text)
    
#     # Remove punctuation and special characters
#     text = re.sub(r'[^a-zA-Z\s]', '', text)
    
#     return text

In [7]:
# Load stopwords
# stop_words = set(stopwords.words('english'))

# # Remove stopwords from tokenized data
# filtered_data = []

# # Print example before
# print("Before data processing and stemming:", tokenized_data[0])

# for sentence in tokenized_data:
#     # Remove stopwords & empty string from the sentence
#     filtered_sentence = [preprocess_text_sklearn(word) for word in sentence if word.lower() not in stop_words]
#     filtered_sentence = [word for word in filtered_sentence if word.strip() != ""]
#     filtered_data.append(filtered_sentence)

# # Print example after 
# print("After data processing and stemming:", filtered_data[0])

Before data processing and stemming: ['@tiniebeany', 'climate', 'change', 'is', 'an', 'interesting', 'hustle', 'as', 'it', 'was', 'global', 'warming', 'but', 'the', 'planet', 'stopped', 'warming', 'for', '15', 'yes', 'while', 'the', 'suv', 'boom']
After data processing and stemming: ['tiniebeany', 'climate', 'change', 'interesting', 'hustle', 'global', 'warming', 'planet', 'stopped', 'warming', 'yes', 'suv', 'boom']


In [2]:
# Hyperparameters
embedding_name = 'glove'  # or 'word2vec'
embedding_dim = 50
num_filters = 50
filter_sizes = [2, 3, 4]
output_dim = 1
dropout = 0.3
learning_rate = 0.0001
num_epochs = 5

In [9]:
# # Load pre-trained word embeddings
# glove = vocab.GloVe(name='6B', dim=embedding_dim)

# # Convert tokenized data to indices using pre-trained embeddings
# indexed_data = [[glove[word].tolist() for word in sentence if word in glove.stoi] for sentence in tokenized_data]

# # Convert indexed data to PyTorch tensors
# data_tensors = [torch.tensor(sentence, dtype=torch.float32) for sentence in indexed_data]

# # Pad sequences to ensure uniform length (optional, depending on your dataset)
# padded_data = nn.utils.rnn.pad_sequence(data_tensors, batch_first=True)

# # Print Example
# print("Example padded data:", padded_data[0])

100%|█████████▉| 399999/400000 [00:13<00:00, 28916.08it/s]


Example padded data: tensor([[ 0.2085,  0.7098, -0.4768,  ...,  0.9483,  0.2803,  0.6592],
        [ 0.0425,  0.1255, -0.0661,  ...,  0.5478,  0.3289,  0.6525],
        [ 0.6185,  0.6425, -0.4655,  ..., -0.2756,  0.3090,  0.4850],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])


In [11]:
# Stop Words
stop_words = set(stopwords.words('english'))

# Word Embedding
word_embedding = vocab.GloVe(name='6B', dim=embedding_dim)

# Tokenization
tokenizer = get_tokenizer("basic_english") 

# Data
data = pd.read_csv("input/twitter_sentiment_data.csv")
message_texts = data["message"]
#sentiment_values = (data["sentiment"]>0.5).astype(int)
sentiment_values = data["sentiment"]

In [12]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class MyDataset(Dataset):
    def __init__(self, tweet_texts, sentiments, tokenizer, stopwords, word_embedding, max_length):
        self.tweet_texts = tweet_texts
        self.sentiments = sentiments
        self.tokenizer = tokenizer
        self.stopwords = stopwords
        self.word_embedding = word_embedding
        self.max_length = max_length
    
    def __len__(self):
        return len(self.tweet_texts)
    
    def __getitem__(self, idx):
        # Get tweet text and sentiment value for the given index
        tweet_text = self.tweet_texts[idx]
        sentiment = self.sentiments[idx]
        
        # Perform tokenization
        tokens = self.tokenizer(tweet_text)
        
        # Perform stop word removal
        tokens = [token for token in tokens if token not in self.stopwords]

        # Pad the tokenized sentence with empty strings
        padded_tokenized_sentence = tokens + [''] * (max_length - len(tokens))
        
        # Perform word embedding
        embeddings = [self.word_embedding[token] for token in padded_tokenized_sentence]
        
        # Convert sentiment to tensor
        sentiment_tensor = torch.tensor([sentiment], dtype=torch.float32)

        padded_embeddings = pad_sequence(embeddings, batch_first=True, padding_value=0.0)
        
        # Convert embeddings to tensor
        embeddings_tensor = torch.tensor(padded_embeddings)
        
        return embeddings_tensor, sentiment_tensor

print("Message Text Shape:", message_texts.shape)
print("Sentiment Value Shape:",sentiment_values.shape)

# Find the maximum length of strings in the Series
max_length = message_texts.str.len().max()

# Create a dataset 
dataset = MyDataset(message_texts, sentiment_values, tokenizer, stop_words, word_embedding, max_length)

batch_size = 64

data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


Message Text Shape: (43943,)
Sentiment Value Shape: (43943,)


In [13]:
# Checking that samples have the same shape before feeding into the model
for i in range(2):  
    inputs, labels = dataset[i]
    print("Sample", i+1)
    print("Inputs dimensions:", inputs.size())
    print("Labels dimensions:", labels.size())
    print()  

Sample 1
Inputs dimensions: torch.Size([623, 50])
Labels dimensions: torch.Size([1])

Sample 2
Inputs dimensions: torch.Size([623, 50])
Labels dimensions: torch.Size([1])



C:\Users\sfip2\AppData\Local\Temp\ipykernel_16336\3645350463.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings_tensor = torch.tensor(padded_embeddings)


In [14]:
# Defining the model
class CNN(nn.Module):
    def __init__(self, embedding_dim, num_filters, filter_sizes, output_dim, dropout):
        super(CNN, self).__init__()
        
        # Convolutional layers
        self.conv_layers = nn.ModuleList([
            nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=fs) 
            for fs in filter_sizes
        ])
        
        # Fully connected layer
        self.fc = nn.Linear(len(filter_sizes) * num_filters, output_dim)
        
        # Dropout layer
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # Convolutional layers
        conv_outputs = [torch.relu(conv(x.transpose(1, 2))) for conv in self.conv_layers]
        
        # Max pooling
        pooled_outputs = [torch.max(conv, 2)[0] for conv in conv_outputs]
        
        # Concatenate pooled outputs
        cat = torch.cat(pooled_outputs, 1)
        
        # Dropout
        cat = self.dropout(cat)
        
        # Fully connected layer
        output = self.fc(cat)
        
        return output

# Initialize model and optimizer
model = CNN(embedding_dim, num_filters, filter_sizes, output_dim, dropout)
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# # Convert padded data to tensor
# input_data = torch.tensor(padded_data, dtype=torch.float32)

In [16]:
# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    # Set the model to training mode
    model.train()
    
    # Initialize the loss for this epoch
    epoch_loss = 0.0
    
    # Iterate over the training dataset in batches
    for inputs, labels in data_loader:  
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(inputs)
        
        # Calculate the loss
        loss = criterion(outputs, labels)
        
        # Backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        
        # Update the parameters (weights) of the model
        optimizer.step()
        
        # Accumulate the loss for this batch
        epoch_loss += loss.item()
    
    # Calculate the average loss for this epoch
    epoch_loss /= len(data_loader)
    
    # Print the average loss for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

C:\Users\sfip2\AppData\Local\Temp\ipykernel_16336\3645350463.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddings_tensor = torch.tensor(padded_embeddings)


KeyboardInterrupt: 